In [6]:
#si-exercise
import pandas as pd
import numpy as np
import scipy as sp

class RegressionModel:
    def __init__(self, x, y, create_intercept=True, regression_type='ols'):
        self.x = x.copy()
        self.y = y
        self.create_intercept = create_intercept
        self.regression_type = regression_type
        self.results = {}
        if self.create_intercept:
            self.add_intercept()

    def add_intercept(self):

        self.x['intercept'] = 1

    def ols_regression(self):

        X = self.x.values
        Y = self.y.values.reshape(-1, 1)

        # OLS estimation
        XtX = np.dot(X.T, X)  # X'X
        XtY = np.dot(X.T, self.y)  # X'y
        beta_hat = sp.linalg.solve(XtX, XtY, assume_a='pos')

        # Predicted values and residuals
        y_hat = np.dot(X, beta_hat)
        residuals = Y - y_hat

        # Calculate degrees of freedom and residual variance
        n = len(self.y)  # number of rows
        k = X.shape[1]  # number of columns
        df = n - k
        residual_variance = np.sum(residuals**2) / df

        # Standard errors
        var_beta_hat = residual_variance * sp.linalg.inv(XtX)  # Variance-covariance matrix of beta_hat
        standard_errors = np.sqrt(np.diag(var_beta_hat))


        t_stats = beta_hat.flatten() / standard_errors
        p_values = 2 * (sp.stats.t.sf(np.abs(t_stats), df))


        for i, col in enumerate(self.x.columns):
            self.results[col] = {
                'coefficient': beta_hat[i][0],
                'standard_error': standard_errors[i],
                't_stat': t_stats[i],
                'p_value': p_values[i]
            }

    def summary(self):
        if not self.results:
            print("No results available. Please run ols_regression first.")
            return

        # Convert results dictionary into a DataFrame for summary
        summary_table = pd.DataFrame(self.results).T
        summary_table.columns = ['coefficient', 'standard_error', 't_stat', 'p_value']
        summary_table.index.name = 'Variable name'

        print(summary_table)
        return summary_table